![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Word and Sentence Embeddings

# Legal Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we use Spark NLP Legal Word (**roberta_embeddings_legal_roberta_base**) and Sentence (**sent_bert_base_uncased_legal**) Embeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

## Import Required Library

In [0]:
from johnsnowlabs import * 
import pandas as pd

# Get sample text

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Legal/data/legal_pca_samples.csv
df = pd.read_csv('legal_pca_samples.csv')

df.head()

Out[2]:

,text,label
0,The fiscal year of the Company (herein called ...,fiscal-year
1,Each of the Borrower and each other member of ...,fiscal-year
2,Purchaser shall pay as the total Purchase Pric...,purchase-price
3,The purchase price to be paid by Purchaser to ...,purchase-price
4,The Guarantor hereby unconditionally and irrev...,guarantee


In [0]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

+--------------------+--------------+
 text| label|
+--------------------+--------------+
The fiscal year o...| fiscal-year|
Each of the Borro...| fiscal-year|
Purchaser shall p...|purchase-price|
The purchase pric...|purchase-price|
The Guarantor her...| guarantee|
The Holding Compa...| guarantee|
GFS will bear its...| expenses|
Each party shall ...| expenses|
Failure by either...| waiver|
Failure of any pa...| waiver|
+--------------------+--------------+

# Sentence Embeddings

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_uncased_legal", "en") \
    .setInputCols("document") \
    .setOutputCol("document_embeddings")

sent_bert_base_uncased_legal download started this may take some time.
Approximate size to download 390.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [0]:
# This class extracts the embeddings from the Spark NLP Annotation object
# from pyspark import ml as ML

from pyspark.ml import Transformer
from pyspark import ml as ML
from pyspark import keyword_only 
from pyspark.sql import functions as F
import pyspark.sql.types as T

class EmbeddingsUDF(
    Transformer, ML.param.shared.HasInputCol,  ML.param.shared.HasOutputCol,
    ML.util.DefaultParamsReadable, ML.util.DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: ML.linalg.Vectors.dense(vs), ML.linalg.VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("document_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [0]:
embeddings_for_pca = EmbeddingsUDF()

In [0]:
DIMENSIONS  = 3

In [0]:
# import pyspark
pca =ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [0]:
# We did all process in one pipeline

pipeline = nlp.Pipeline().setStages([document_assembler, embeddings, embeddings_for_pca, pca])

In [0]:
pipeline.getStages()

Out[10]: [DocumentAssembler_56e23f0b5a16,
 BERT_SENTENCE_EMBEDDINGS_dae49a767331,
 EmbeddingsUDF_492afe97518a,
 PCA_64b87608dfe6]

In [0]:
model = pipeline.fit(sdf)

In [0]:
result = model.transform(sdf)

In [0]:
result.select('pca_features', 'label').show(truncate=False)

+------------------------------------------------------------+--------------+
pca_features |label |
+------------------------------------------------------------+--------------+
[-11.772477034904608,-3.189962369128527,4.491137648916197] |fiscal-year |
[-11.401123661713259,-3.7697457285371905,3.240525435639572] |fiscal-year |
[-4.783320443150728,-0.4942616667908408,2.8697682040224937] |purchase-price|
[-5.455994051962746,-1.341260074480057,3.331715785253001] |purchase-price|
[-8.841677620092424,-1.8203607493804024,0.13390319958401725]|guarantee |
[-11.532878941614142,-2.4995702800175503,0.8141281936195685]|guarantee |
[-5.7317047066223745,-3.8158763820398773,3.5135625053171022]|expenses |
[-3.8010565515445687,-4.3453814052287045,1.6774177984068788]|expenses |
[-6.783557847085066,-5.815457902520404,3.2361531755360478] |waiver |
[-7.139204069336535,-6.340446439270053,1.2347138234017956] |waiver |
+------------------------------------------------------------+--------------+

In [0]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

Out[14]:

,pca_features,label
0,"[-11.772477034904608, -3.189962369128527, 4.49...",fiscal-year
1,"[-11.401123661713259, -3.7697457285371905, 3.2...",fiscal-year
2,"[-4.783320443150728, -0.4942616667908408, 2.86...",purchase-price
3,"[-5.455994051962746, -1.341260074480057, 3.331...",purchase-price
4,"[-8.841677620092424, -1.8203607493804024, 0.13...",guarantee
5,"[-11.532878941614142, -2.4995702800175503, 0.8...",guarantee
6,"[-5.7317047066223745, -3.8158763820398773, 3.5...",expenses
7,"[-3.8010565515445687, -4.3453814052287045, 1.6...",expenses
8,"[-6.783557847085066, -5.815457902520404, 3.236...",waiver
9,"[-7.139204069336535, -6.340446439270053, 1.234...",waiver


In [0]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

Out[15]:

,x,y,z,label
0,-11.772477,-3.189962,4.491138,fiscal-year
1,-11.401124,-3.769746,3.240525,fiscal-year
2,-4.783320,-0.494262,2.869768,purchase-price
3,-5.455994,-1.341260,3.331716,purchase-price
4,-8.841678,-1.820361,0.133903,guarantee
5,-11.532879,-2.499570,0.814128,guarantee
6,-5.731705,-3.815876,3.513563,expenses
7,-3.801057,-4.345381,1.677418,expenses
8,-6.783558,-5.815458,3.236153,waiver
9,-7.139204,-6.340446,1.234714,waiver


In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)

fig.show()

# Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before.

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols("document")\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"])\
    .setOutputCol("document_embeddings")

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]

In [0]:
# Firstly we splitted the pipeline in two to get all token embeddings

pipeline = nlp.Pipeline().setStages([document_assembler, tokenizer, embeddings])

In [0]:
model = pipeline.fit(sdf)

In [0]:
result = model.transform(sdf)

In [0]:
from pyspark.sql import functions as F

result_df = result.select("label", F.explode(F.arrays_zip(result.token.result, result.document_embeddings.embeddings)).alias("cols"))\
                   .select(F.expr("cols['0']").alias("token"),
                           F.expr("cols['1']").alias("embeddings"),
                           "label")

result_df.show(truncate = 80)


+--------+--------------------------------------------------------------------------------+-----------+
 token| embeddings| label|
+--------+--------------------------------------------------------------------------------+-----------+
 The|[-0.19058317, 0.029071994, 0.13235612, 0.19562241, 0.77783275, 0.28990918, -0...|fiscal-year|
 fiscal|[-0.19621778, 0.14509659, 0.23111394, -0.50601685, -0.38397115, -0.16950977, ...|fiscal-year|
 year|[0.080063045, 0.22008333, 0.23202503, -0.44197246, 0.5893604, -0.2369217, 0.1...|fiscal-year|
 of|[-0.14090301, 0.15613586, 0.24000236, -0.24493216, 0.8977561, 0.4878103, 0.09...|fiscal-year|
 the|[-0.060954362, -0.082320824, 0.31499287, 0.12840028, -0.014586113, 0.9788808,...|fiscal-year|
 Company|[-0.060746957, 0.2748839, 0.071460865, -0.39569914, 0.7331539, 0.80515695, 0....|fiscal-year|
 (|[0.11098497, -0.23188403, 0.11235523, 0.07458286, 0.86609787, -0.11237905, -0...|fiscal-year|
 herein|[-0.08064833, -0.058289964, 0.0014471039, -0.014948182, 0.27398282, -0.203155...|fiscal-year|
 called|[0.053557705, 0.15397239, 0.06047485, 0.4370787, 1.2082455, 0.19268191, 0.225...|fiscal-year|
 the|[-0.08710302, 0.07147782, 0.15197298, 0.064896256, -0.08188157, 0.29937837, -...|fiscal-year|
 "|[0.055207413, 0.19191724, 0.009991523, -0.002567254, 1.0418495, 0.21364397, -...|fiscal-year|
 fiscal|[-0.14709531, 0.017821867, 0.27213293, -0.48752296, -0.48709142, -0.22528103,...|fiscal-year|
 year|[0.09589189, -0.018595183, 0.24576819, -0.52658284, 0.23036416, -0.16599034, ...|fiscal-year|
 ")|[0.078798935, -0.059295893, -0.09431608, 0.22301352, 0.49477506, -0.19168545,...|fiscal-year|
 shall|[-0.18689284, -0.030555159, 0.2499313, -0.08885675, -0.1576013, -0.33531103, ...|fiscal-year|
 end|[0.1565781, -0.16092362, 0.2524219, 0.055000514, -2.078185, -0.212337, 0.0180...|fiscal-year|
 on|[0.37850145, 0.450114, 0.36284637, 0.48866224, 0.7129804, -0.018029183, 0.253...|fiscal-year|
December|[0.46872857, 0.040406935, -0.021835145, 0.15274885, 1.0853437, 0.26045802, 0....|fiscal-year|
 31|[0.20654203, 0.15854512, 0.18410331, 0.081838444, 0.73909575, -0.0188054, -0....|fiscal-year|
 of|[0.043495826, -0.115976885, 0.090747535, -0.10574537, 1.0775031, 0.432438, 0....|fiscal-year|
+--------+--------------------------------------------------------------------------------+-----------+
only showing top 20 rows

In [0]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [0]:
embeddings_for_pca = WordEmbeddingsUDF()

In [0]:
DIMENSIONS  = 3

In [0]:
# import pyspark
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

## Full Spark NLP + Spark MLLib pipeline

In [0]:
# We run the second part of the pipeline

pipeline = nlp.Pipeline().setStages([embeddings_for_pca, pca])


In [0]:
model = pipeline.fit(result_df)

In [0]:
result = model.transform(result_df)

In [0]:
result.select("token", "embeddings", "pca_features", "label").show(truncate = 60)

+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
 token| embeddings| pca_features| label|
+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
 The|[-0.19058329, 0.029071923, 0.13235596, 0.19562289, 0.7778...| [6.574773139044961,7.65286502110281,-0.9220370651507019]|fiscal-year|
 fiscal|[-0.1962179, 0.14509675, 0.23111376, -0.5060164, -0.38397...|[5.227395982797811,-0.06940068997057545,-1.3976507425299076]|fiscal-year|
 year|[0.08006279, 0.22008295, 0.23202541, -0.44197226, 0.58936...| [9.185602029229745,1.0014820713259496,1.0998380196513011]|fiscal-year|
 of|[-0.14090335, 0.1561357, 0.24000221, -0.2449323, 0.897755...| [6.397932523970772,3.1917363167944006,-1.0405452759492786]|fiscal-year|
 the|[-0.060954727, -0.08232096, 0.31499285, 0.12840018, -0.01...| [7.772305113086351,8.005479063316065,-2.2717362630055336]|fiscal-year|
 Company|[-0.060747035, 0.27488366, 0.071460515, -0.39569908, 0.73...| [3.2013630953187744,3.9378576440870865,-2.4062405424512923]|fiscal-year|
 (|[0.11098501, -0.23188378, 0.112354875, 0.074582785, 0.866...| [3.2695715452218588,1.5800771113783936,2.126949936572752]|fiscal-year|
 herein|[-0.08064853, -0.058289804, 0.0014475286, -0.014947139, 0...| [5.247827854971892,0.5408455644434723,-0.47203559635162484]|fiscal-year|
 called|[0.053557344, 0.15397242, 0.0604746, 0.43707865, 1.208246...| [4.847899462787553,1.4702135922698991,0.5524350334979414]|fiscal-year|
 the|[-0.08710282, 0.07147787, 0.1519727, 0.06489649, -0.08188...| [6.271756003613809,5.412340195850786,-0.679607480081284]|fiscal-year|
 "|[0.055207603, 0.19191724, 0.009991426, -0.002567172, 1.04...| [2.4479703147472436,1.5318544006726906,-0.1030593364655475]|fiscal-year|
 fiscal|[-0.1470949, 0.017821616, 0.27213296, -0.48752207, -0.487...| [5.30223837521482,-0.15669390686741477,-1.4091701159743826]|fiscal-year|
 year|[0.0958918, -0.01859492, 0.24576774, -0.52658165, 0.23036...| [9.220589693373142,1.3564950447736226,1.429935281900738]|fiscal-year|
 ")|[0.07879856, -0.059296045, -0.09431649, 0.22301316, 0.494...| [1.942833580266059,1.5016631671533165,2.0069552567624473]|fiscal-year|
 shall|[-0.18689293, -0.030555077, 0.24993157, -0.088856995, -0....| [5.095635900683648,6.43895215927085,0.7663093530496257]|fiscal-year|
 end|[0.15657794, -0.16092344, 0.25242174, 0.05500093, -2.0781...| [6.942599536574128,1.3926476311989315,-0.3101340721290469]|fiscal-year|
 on|[0.37850103, 0.4501142, 0.36284617, 0.48866212, 0.7129792...| [7.604270660136931,3.464275325349722,1.3114427731214227]|fiscal-year|
December|[0.46872932, 0.040407117, -0.021835271, 0.15274842, 1.085...| [6.591850803302487,2.6569165891621243,-3.111520074252188]|fiscal-year|
 31|[0.20654224, 0.15854545, 0.18410304, 0.081837706, 0.73909...| [4.474305556855789,1.8455638482757022,0.5983487945638217]|fiscal-year|
 of|[0.04349587, -0.11597654, 0.090747386, -0.10574573, 1.077...| [6.01818652860658,3.19892131146054,-0.014780620079350411]|fiscal-year|
+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
only showing top 20 rows

In [0]:
df = result.select('token', 'pca_features',  'label').toPandas()

df

Out[30]:

,token,pca_features,label
0,The,"[6.574773139044961, 7.65286502110281, -0.92203...",fiscal-year
1,fiscal,"[5.227395982797811, -0.06940068997057545, -1.3...",fiscal-year
2,year,"[9.185602029229745, 1.0014820713259496, 1.0998...",fiscal-year
3,of,"[6.397932523970772, 3.1917363167944006, -1.040...",fiscal-year
4,the,"[7.772305113086351, 8.005479063316065, -2.2717...",fiscal-year
...,...,...,...
672,constitute,"[6.129947148631695, -1.5722397283753062, -0.58...",waiver
673,a,"[8.76467597454233, 1.0030852667560528, 0.45322...",waiver
674,continuing,"[4.1503600980893305, -0.7722882042241939, -1.1...",waiver
675,waiver,"[7.824426762455292, -1.4749742620145976, -3.41...",waiver


In [0]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "x", "y", "z", "label"]]

df

Out[31]:

,token,x,y,z,label
0,The,6.574773,7.652865,-0.922037,fiscal-year
1,fiscal,5.227396,-0.069401,-1.397651,fiscal-year
2,year,9.185602,1.001482,1.099838,fiscal-year
3,of,6.397933,3.191736,-1.040545,fiscal-year
4,the,7.772305,8.005479,-2.271736,fiscal-year
...,...,...,...,...,...
672,constitute,6.129947,-1.572240,-0.582971,waiver
673,a,8.764676,1.003085,0.453228,waiver
674,continuing,4.150360,-0.772288,-1.152194,waiver
675,waiver,7.824427,-1.474974,-3.411841,waiver


In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()

That chart is super cool because you can see how the same token gets different embeddings depending on the context.